In [4]:
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
events = pd.read_csv('../data/events.csv.gzip', compression='gzip',\
                     parse_dates=['date'],\
                     dtype={'event_id':'object',\
                            'application_id':'category',\
                            'ref_type':'int64',\
                            'ref_hash':'int64',\
                            'attributed':'bool',\
                            'device_countrycode':'category',\
                            'device_os_version':'category',\
                            'device_brand':'category',\
                            'device_model':'category',\
                            'device_city':'category',\
                            'carrier':'category',\
                            'event_uuid':'object',\
                            'device_os':'category',\
                            'wifi':'bool',\
                            'connection_type':'category'\
                           })

In [ ]:
events.describe(include='all').transpose()

**device_country_code** es unico por lo que no aporta informacion para segmentar los datos.
**event_uuid** es clave unica para cada registro

In [ ]:
events.shape

In [3]:
events.info()

NameError: name 'events' is not defined

In [ ]:
events.sort_values(by=['ref_hash','application_id','date'], inplace=True)

In [ ]:
events['next_date'] = events.groupby(['ref_hash','application_id'], as_index = False, sort=False)['date']\
                                              .transform(lambda x: x.shift(-1))

In [ ]:
events['next_event'] = events.groupby(['ref_hash','application_id'], as_index = False, sort=False)['event_id']\
                                              .transform(lambda x: x.shift(-1))

In [ ]:
events['next_event_2'] = events.groupby(['ref_hash','application_id'], as_index = False, sort=False)['event_id']\
                                              .transform(lambda x: x.shift(-2))

In [ ]:
events = events.loc[(~events['next_date'].isnull())]
events['secs_to_next'] = (events['next_date'] - events['date'])\
                                        .transform(lambda x: round(x.total_seconds(), 2))
events['hour'] = events['date'].dt.hour
events['is_weekend'] = events['date'].dt.weekday >= 5

## Tiempo entre eventos

In [ ]:
plt = events['secs_to_next'].loc[((events['secs_to_next'] < (10)) & (events['secs_to_next'] > 0))].hist(bins=200, density=True)
plt.set_title("Tiempo entre eventos")
plt.set_xlabel("Tiempo (Seg.)")

Se pueden ver dos picos, el primero ocurre muy cerca del 0, por lo que parece que se debe a dos eventos que ocurren en paralelo. 
Lo interesante es que practicamente todos los eventos ocurren en intervalos de no mas de 3 segundos. Tomamos 5 segundos como intervalo para no considerar que los eventos estan relacionado con el evento siguiente

## Cual es la secuencia de eventos mas comun? 

Para este analisis se separan las secuencias de eventos diferentes, y se cuentan las secuencias. 

Consideraciones: 
* Si se demora mas de 5 segundos entre eventos se los considera no relacionados.
* Solo se consideran secuencias de eventos distintos, si el usuario realiza el mismo evento varias veces no se tiene en cuenta

In [ ]:
events['n'] = 1
e_list = events.loc[((events['secs_to_next'] < 5) & (events['event_id'] != events['next_event']))]

In [ ]:
cant_sec = e_list.groupby(['event_id','next_event'], as_index=False)['n'].sum().dropna()

In [ ]:
plt = cant_sec.sort_values(by='n',ascending=False).head(20).set_index(['event_id','next_event'])['n'].plot.bar()
plt.set_title('Par de eventos mas comunes')
plt.set_xlabel('Secuencia de 2 eventos')

Si considero que luego de 5 segundos sin interaccion, se dejo de usar la aplicacion, llamo a este evento "Fin":

In [ ]:
events['next_event_cfin'] = events['next_event']
events.loc[(events['secs_to_next'] > 5), 'next_event_cfin'] = 'Fin'

In [ ]:
plt = events.groupby(['event_id','next_event_cfin'], as_index=False)['n'].sum().dropna().sort_values(by='n',ascending=False).head(20).set_index(['event_id','next_event_cfin'])['n'].plot.bar()
plt.set_title('Par de eventos mas comunes, con fin de uso')
plt.set_xlabel('Secuencia de 2 eventos')

### Secuencia de 3 eventos:

In [ ]:
e_list = events.loc[((events['secs_to_next'] < 5) & (events['event_id'] != events['next_event']) & (events['next_event'] != events['next_event_2']))]
cant_sec = e_list.groupby(['event_id','next_event','next_event_2'], as_index=False)['n'].sum().dropna()
plt = cant_sec.sort_values(by='n',ascending=False).head(20).set_index(['event_id','next_event','next_event_2'])['n'].plot.bar()
plt.set_title('Terna de eventos mas comunes')
plt.set_xlabel('Secuencia de 3 eventos')